# LANDSAT8 - Downloading band data for multiple dates
This code has been specifically written for Landsat ARD data. The idea is that you provide the specific dates that you wish to process over a given area (defined by the polygon), that you have already sussed for cloud QA. Be warned, datasets are big.  Always try to stay about 1gb under your threshold to ensure your work gets processed.

In [1]:
%matplotlib inline

import datacube
import geopandas as gpd
import numpy as np
from datacube.utils import geometry
from datacube.utils.cog import write_cog

import sys
sys.path.insert(1, '../Tools/')
from dea_tools.datahandling import load_ard
from dea_tools.plotting import map_shapefile
from dea_tools.bandindices import calculate_indices

### Connect to the datacube

In [2]:
# Temporary solution to account for Collection 3 data being in a different
# database on the NCI
try:
    dc = datacube.Datacube(app='Analyse_multiple_polygons', env='c3-samples')
except:
    dc = datacube.Datacube(app='Analyse_multiple_polygons')

## Analysis parameters

* `time_list` : Enter dates, in units YYYY-MM-DD, for each target date  e.g. `'2019-01-01'`
* `vector_file` : A path to a vector file (ESRI Shapefile or GeoJSON). Code below presumes your vector to live in a directory called 'vectors' located in the same folder as this notebook. Please ensure it is projected to Albers Equal Area projection
* `attribute_col` : A column in the vector file used to label the output `xarray` datasets containing satellite images. Each row of this column should have a unique identifier
* `products` : Here we are using Landsat8 only
* `measurements` : A list of band names to load from the satellite product e.g. `['nbart_red', 'nbart_green']`
* `resolution` : The spatial resolution of the loaded satellite data e.g. for Landsat, this is `(-30, 30)`
* `output_crs` : The coordinate reference system/map projection to load data into, e.g. `'EPSG:3577'` to load data in the Albers Equal Area projection
* `align` : How to align the x, y coordinates respect to each pixel. Landsat Collection 3 should be centre aligned `align = (15, 15)` if data is loaded in its native UTM zone projection, e.g. `'EPSG:32756'` 

In [3]:
# Enter specific dates
time_list = (
"2022-05-13",
"2022-05-13"
            )

# Converts the list to a list of tuples (which the code needs)
time_range = [(time_list[i],time_list[i]) for i in range(0,len(time_list))]


# Extents of the images to download (note: Albers 3577). These vector files need to be first uploaded to the sandbox directory
vector_file = './vectors/ningaloo_processing_aoi.shp'

# give attribute name to id outputs
attribute_col = 'aoi' 

# These are the Landsat 8 product descriptions, do not change
products = ['ga_ls8c_ard_3'] 

measurements = ['nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir'] # choose only the bands you need to save space
resolution = (-30, 30)
output_crs = 'EPSG:3577'
align = (0, 0)

In [4]:
# read in original vectors
gdf = gpd.read_file(vector_file)
geom = geometry.Geometry(gdf.iloc[0].geometry.__geo_interface__, 
                          crs=geometry.CRS(output_crs))

# un-comment if you wish to visualise the polygon
map_shapefile(gdf, attribute=attribute_col)


/home/jovyan/Bart/../Tools/dea_tools/plotting.py:418: FutureWarning: The `map_shapefile` function is deprecated, and will be removed from future versions of `dea-tools`. Please use Geopanda's built-in `.explore` functionality instead.
  warnings.warn("The `map_shapefile` function is deprecated, and will "


Label(value='')

Map(center=[-21.960797363881863, 113.93438124230774], controls=(ZoomControl(options=['position', 'zoom_in_text…

In [5]:
# iterating over tuple elements
for time_step in time_range:
    
    print(time_step)
      
    # Create a datacube dictionary query object    
    query = {'time': time_step,
        'geopolygon': geom,
        'measurements': measurements,
        'resolution': resolution,
        'output_crs': output_crs,
        'align': align,
        }
    
    try: # The try except clauses stop the script failing if the image does not excist in the sandbox but it does in glovis for example, if goes to the next image
        # Load the image(s)
        ds = load_ard(dc=dc, 
                       products=products,
                       mask_pixel_quality=False,            
                       group_by='solar_day',
                       **query)
        
 
        ### Create output file variables and write to GeoTiff

        # Get time and site variable for naming
        time = ds.groupby('time')
        site = gdf.loc[0 ,attribute_col]
        
        # Select a single time-slice of each new index band
        ds = ds.isel(time=0).to_array()

        # Write GeoTIFF
        name = f'{site}_{str(time)[57:67]}_L8_multiband.tif'
        
        print("\nWriting image: ...")
        write_cog(geo_im=ds,
                  fname=name,
                  overwrite=True) # outputs in ipynp dir
        
        print(name)
        

    except (ValueError, KeyError): # I do not know how to use error exception, but this prints the details of the image that the sandbox does not have.
        
        print("\nThe image combination of below variables not accessable:",
              "----------------------------------------------------------",
              "Time_step: {}".format(time_step),
              "Product: {}".format(products),
              "ValueError: {}".format(ValueError),
              "KeyError: {}".format(KeyError),
              sep='\n')
        
        pass
    finally:
        print("\nNext image...\n")
        
print("Fin!")       

('2022-05-13', '2022-05-13')
Finding datasets
    ga_ls8c_ard_3
Loading 1 time steps

Writing image: ...
ningaloo_2022-05-13_L8_multiband.tif

Next image...

('2022-05-13', '2022-05-13')
Finding datasets
    ga_ls8c_ard_3
Loading 1 time steps

Writing image: ...
ningaloo_2022-05-13_L8_multiband.tif

Next image...

Fin!


In [6]:
# Tarball - CHANGE directory name to yours
# remove block quotes (''') from beginning and end and 'play' like all cells above if they are there
# Tar ball the whole directory with a file name variable of the name of the last file created

# print(name2)
# tarName = "1" + name2[:-4] # in the shell the first char is drop off!!!
# print(tarName)
# ! tar -czvf ${tarName}.tar.gz '/home/jovyan/testing/'

T51HUE_test_2020-05-29_L8_NBR_10m.tif
1T51HUE_test_2020-05-29_L8_NBR_10m
tar: Removing leading `/' from member names
/home/jovyan/testing/
/home/jovyan/testing/T51HUE_test_2020-05-13_L8_NBR_10m.tif
/home/jovyan/testing/T51HUE_test_2020-05-13_L8_NDVI_10m.tif
/home/jovyan/testing/T51HUE_test_2020-05-29_L8_NBR_10m.tif
/home/jovyan/testing/SENTINEL_select_dates_index_download.ipynb
/home/jovyan/testing/GL_Landsat_scene_download_20210701_Loop_working.ipynb
/home/jovyan/testing/vectors/
/home/jovyan/testing/vectors/AOI_alb.prj
/home/jovyan/testing/vectors/AOI_alb.shp
/home/jovyan/testing/vectors/AOI_alb.shx
/home/jovyan/testing/vectors/.ipynb_checkpoints/
/home/jovyan/testing/vectors/AOI_alb.dbf
/home/jovyan/testing/Landsat_select_dates_index_download-Copy1.ipynb
/home/jovyan/testing/.ipynb_checkpoints/
/home/jovyan/testing/.ipynb_checkpoints/Landsat_select_dates_index_download-Copy1-checkpoint.ipynb
/home/jovyan/testing/.ipynb_checkpoints/SENTINEL_select_dates_index_download-checkpoint.ipyn